In [14]:
url1 = (r'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/latest/owid-covid-latest.json')

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import requests, pandas as pd
from bs4 import BeautifulSoup
import urllib.request

In [67]:

#This is live data from the our world in data github. This is the csv they put on the website. Covid19 data.
data=pd.read_json(url1)
data1 = pd.DataFrame(data).T
data1 = data1.astype(str)
data1 = data1[data1 != 'None']
data1 = data1['total_deaths'].astype(float)


In [76]:
#Attempted to create the dataframe from the url but I got a forbidden 403 error.
url2 = ('https://apps.who.int/gho/athena/api/GHO/HWF_000.json?filter=YEAR:2019&filter=text')
url2

'https://apps.who.int/gho/athena/api/GHO/HWF_000.json?filter=YEAR:2019&filter=text'

In [81]:
#Printed encoding of csv file to read the csv.
with open(r'C:\Users\vw191\OneDrive\Documents\GitHub\107SBakst.github.io\data\who_doctors.csv') as f:
        print(f.encoding)

cp1252


In [94]:
#Created the dataframe and cleaned the location column.
data2 = pd.read_csv(r'C:\Users\vw191\OneDrive\Documents\GitHub\107SBakst.github.io\data\who_doctors.csv', encoding ="cp1252")
data2 = data2[data2['Period'] == 2018]
data2['Location'] = data2['Location'].replace('Iran (Islamic Republic of)', 'Iran')
data2['Location'] = data2['Location'].replace('CÃ´te dâ€™Ivoire', 'Ivory Coast')
data2['Location'] = data2['Location'].replace('United States of America', 'United States')
data2['Location'] = data2['Location'].replace('United Kingdom of Great Britain and Northern Ireland', 'United Kingdom')
data2['Location'] = data2['Location'].replace('Russian Federation', 'Russia')
data2

,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified
41,HWF_0001,"Medical doctors (per 10,000)",numeric,AFR,Africa,Country,MWI,Malawi,Year,2018,...,NaN,NaN,NaN,NaN,NaN,0.36,NaN,NaN,EN,2021-02-17T00:00:00.000Z
42,HWF_0001,"Medical doctors (per 10,000)",numeric,WPR,Western Pacific,Country,PNG,Papua New Guinea,Year,2018,...,NaN,NaN,NaN,NaN,NaN,0.70,NaN,NaN,EN,2021-02-17T00:00:00.000Z
43,HWF_0001,"Medical doctors (per 10,000)",numeric,AFR,Africa,Country,SLE,Sierra Leone,Year,2018,...,NaN,NaN,NaN,NaN,NaN,0.74,NaN,NaN,EN,2021-02-17T00:00:00.000Z
44,HWF_0001,"Medical doctors (per 10,000)",numeric,AFR,Africa,Country,ETH,Ethiopia,Year,2018,...,NaN,NaN,NaN,NaN,NaN,0.77,NaN,NaN,EN,2021-02-17T00:00:00.000Z
45,HWF_0001,"Medical doctors (per 10,000)",numeric,AFR,Africa,Country,TGO,Togo,Year,2018,...,NaN,NaN,NaN,NaN,NaN,0.77,NaN,NaN,EN,2021-02-17T00:00:00.000Z
46,HWF_0001,"Medical doctors (per 10,000)",numeric,AFR,Africa,Country,BEN,Benin,Year,2018,...,NaN,NaN,NaN,NaN,NaN,0.79,NaN,NaN,EN,2021-02-17T00:00:00.000Z
47,HWF_0001,"Medical doctors (per 10,000)",numeric,AFR,Africa,Country,MOZ,Mozambique,Year,2018,...,NaN,NaN,NaN,NaN,NaN,0.84,NaN,NaN,EN,2021-02-17T00:00:00.000Z
48,HWF_0001,"Medical doctors (per 10,000)",numeric,AFR,Africa,Country,BFA,Burkina Faso,Year,2018,...,NaN,NaN,NaN,NaN,NaN,0.85,NaN,NaN,EN,2021-02-17T00:00:00.000Z
49,HWF_0001,"Medical doctors (per 10,000)",numeric,AFR,Africa,Country,GHA,Ghana,Year,2018,...,NaN,NaN,NaN,NaN,NaN,0.96,NaN,NaN,EN,2021-02-17T00:00:00.000Z
50,HWF_0001,"Medical doctors (per 10,000)",numeric,AFR,Africa,Country,MLI,Mali,Year,2018,...,NaN,NaN,NaN,NaN,NaN,1.29,NaN,NaN,EN,2021-02-17T00:00:00.000Z
